## 1.截断
对于每一条的text信息，将其按照500词及以内为一段进行截取，要保证截取时上一句正好完整结束，  
对于初始长度不足500的text记录，直接保留其text和label存入新json文件，  
对于长度大于500的text记录，舍弃掉最后长度不足200词的尾段，且label保持与截取前的相同。  
示例如下：一个label为clean的长度为2100词的text，其处理为4个长度在500词左右的label为clean的text

In [12]:
import json

def process_texts(input_file, output_file):
    with open(input_file, 'r', encoding='utf-8') as file:
        data = json.load(file)
    
    processed_data = []

    for entry in data:
        text = entry['text']
        label = entry['label']
        words = text.split()
        num_words = len(words)

        if num_words < 500:
            # 如果文本长度不足500词，直接添加到处理后的数据中
            processed_data.append({'text': text, 'label': label})
        else:
            # 按500词分段
            i = 0
            while i < num_words:
                next_limit = i + 500 if i + 500 < num_words else num_words
                next_chunk = ' '.join(words[i:next_limit])
                
                if next_limit - i < 500:
                    if next_limit - i < 200:
                        break  # 如果剩余词数小于200，则丢弃
                    else:
                        # 如果剩余词数在200到500之间，则保留
                        processed_data.append({'text': next_chunk, 'label': label})
                        break
                
                # 寻找500词之后的句子结束位置
                end_index = next_chunk.rfind('.')
                if end_index == -1:
                    end_index = next_chunk.rfind('!')
                if end_index == -1:
                    end_index = next_chunk.rfind('?')
                
                if end_index != -1:
                    # 确保只包括完整的句子
                    next_chunk = next_chunk[:end_index+1]
                    processed_data.append({'text': next_chunk, 'label': label})
                
                # 更新索引位置，继续下一个500词段
                i += end_index + 1 if end_index != -1 else 500

    # 将处理后的数据保存到新的JSON文件中
    with open(output_file, 'w', encoding='utf-8') as file:
        json.dump(processed_data, file, ensure_ascii=False, indent=4)

# 使用该函数处理数据
process_texts('dataset/valid.json', 'dataset/preprocessed_valid.json')

print("cut into 500 success!")

cut into 500 success!


## 2.处理补全+打分
对于上述程序生成的经过预处理的原文件，对每条text，将其按照某比例ratio进行截断，需保证截断处是完整句子的结束，将截断处之前的部分截取出来，作为text用detect model进行补全，补全至500词左右形成新的text。对于新的text，其label不再是clean或dirty，而是一个介于0与1之间的value值，value值生成规则为：原有text的label若为clean则ori_val=0，若为dirty则ori_val=1，新生成的值val=ori_val*ratio + const*(1-ratio)*softmax() ，其中const是一个人为设定的常数0.1，ratio是截断比例，softmax是非线性的激发函数

In [11]:
import json
import numpy as np
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM

# 载入经过预处理的数据
with open('dataset/valid.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

# 设置detect model
# tokenizer = AutoTokenizer.from_pretrained("detected_model")
# model = AutoModelForCausalLM.from_pretrained("detected_model", torch_dtype=torch.bfloat16)
# text_generator = pipeline('text-generation', model=model, tokenizer=tokenizer, max_length=500)

# 定义softmax函数
def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

# 定义常数和比例
const = 0.1
ratio = 0.25  # 可以根据需要调整比例

processed_data = []

for entry in data:
    text = entry['text']
    label = entry['label']

    # 根据label设置原始value
    ori_val = 0 if label == 'clean' else 1

    # 按照比例截断
    words = text.split()
    cut_index = int(len(words) * ratio)
    truncated_text = ' '.join(words[:cut_index])

    # 确保在完整的句子结束处截断
    end_punctuation = max(truncated_text.rfind('.'), truncated_text.rfind('!'), truncated_text.rfind('?'))
    if end_punctuation != -1:
        truncated_text = truncated_text[:end_punctuation+1]

    # 使用模型补全文本
    # generated_text = text_generator(truncated_text, max_length=500)[0]['generated_text']
    generated_text = truncated_text
    
    # 计算新的value
    new_val = ori_val * ratio + const * (1 - ratio) * softmax(np.array([1]))[0]  # 简化了softmax的使用

    # 添加到新的数据集
    processed_data.append({
        'text': generated_text,
        'value': new_val
    })

# output_filename = f'train_{ratio:.2f}.json'
output_filename = 'dataset/0.25_complete_train.json'

# 保存新生成的数据
with open(output_filename, 'w', encoding='utf-8') as file:
    json.dump(processed_data, file, ensure_ascii=False, indent=4)

print("build data success!")

build data success!
